In [29]:
#Given two images, image 2 will be transformed to match the geometry of image 1.
import itk
import nibabel as nib
import numpy as np
import nrrd

ImagePath2 = "Z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-42-Spiromics-67503688/Markups/88-Adam/S-Final-2-label.nrrd"
ImagePath1 = "Z:/E-Kelly-VidaSegmentals&Sublobes/3- 20Spiromics-10ForKelly-ForERS23/Case42-IA210257_H-17900_20190805_125316_20230524_121115_export/ZUNU_vida-airtree-recentered.nii.gz"
#Image 2 is the airway while image 1 is the arterial tree. We will take image 2 and resample it using image 1 geometry
#Image 1 is an nrrd and Image 2 is a nii.gz

# Read the NRRD image using ITK NOTE LPS IS USED INSTEAD OF RAS
im1 = itk.imread(ImagePath1, itk.SS)
origin1 = im1.GetOrigin()  # Get image origin
spacing1 = im1.GetSpacing()  # Get pixel spacing
size1 = im1.GetLargestPossibleRegion().GetSize()  # Get image size

im2 = itk.imread(ImagePath2, itk.SS)
origin2 = im2.GetOrigin()  # Get image origin
spacing2 = im2.GetSpacing()  # Get pixel spacing
direction2 = im2.GetDirection()  # Get image direction (orientation)
size2 = im2.GetLargestPossibleRegion().GetSize()  # Get image size

#Find minimum and maximum pixel values of image 2 for thresholding purposes.
min_max_calculator = itk.MinimumMaximumImageCalculator[itk.Image[itk.SS, 3]].New(Image=im2)
min_max_calculator.Compute()
min = min_max_calculator.GetMinimum()
upperThresh = min_max_calculator.GetMaximum() + 1
lowerThresh = int(min_max_calculator.GetMaximum() / 2)


In [28]:
upperThresh

3

In [30]:
import os

# Resample im2 to match im1's origin, spacing, and size
resampler = itk.ResampleImageFilter.New(Input=im2)
resampler.SetSize([int(x) for x in size1])
resampler.SetOutputOrigin(origin1)
resampler.SetOutputSpacing(spacing1)

# Set identity transform
transform = itk.IdentityTransform[itk.D, 3].New()
resampler.SetTransform(transform)

resampler.UseReferenceImageOn()
resampler.SetReferenceImage(im1)
resampler.Update()

# Threshold the resampled image to create a binary mask
threshold_filter = itk.BinaryThresholdImageFilter.New(Input=resampler.GetOutput())
threshold_filter.SetLowerThreshold(lowerThresh)  # Threshold value, adjust if needed
threshold_filter.SetUpperThreshold(upperThresh)  # Upper threshold value, adjust if needed
threshold_filter.SetInsideValue(1)
threshold_filter.SetOutsideValue(0)
threshold_filter.Update()

outputFile = "C:/Users/nichmoore/Documents/GitHub/RA4-Project/figures/airtreeResampledSwapped.nii.gz"

if os.path.exists(outputFile):
    os.remove(outputFile)

# Save the resampled image as NIfTI
airtreeResampled = threshold_filter.GetOutput()
itk.imwrite(airtreeResampled, outputFile)

In [ ]:
#Framework for function itself

def TransformImageGeometry(image1, image2):
    origin1 = image1.GetOrigin()  # Get image origin
    spacing1 = image1.GetSpacing()  # Get pixel spacing
    size1 = image1.GetLargestPossibleRegion().GetSize()  # Get image size
    origin2 = image2.GetOrigin()  # Get image origin
    spacing2 = image2.GetSpacing()  # Get pixel spacing
    direction2 = image2.GetDirection()  # Get image direction (orientation)
    size2 = image2.GetLargestPossibleRegion().GetSize()  # Get image size

    min_max_calculator = itk.MinimumMaximumImageCalculator[itk.Image[itk.SS, 3]].New(Image=image2)
    min_max_calculator.Compute()
    upperThresh = min_max_calculator.GetMaximum() + 1
    lowerThresh = int(min_max_calculator.GetMaximum() / 2)

    # Resample im2 to match im1's origin, spacing, and size
    resampler = itk.ResampleImageFilter.New(Input=image2)
    resampler.SetSize([int(x) for x in size1])
    resampler.SetOutputOrigin(origin1)
    resampler.SetOutputSpacing(spacing1)

    # Set identity transform
    transform = itk.IdentityTransform[itk.D, 3].New()
    resampler.SetTransform(transform)

    resampler.UseReferenceImageOn()
    resampler.SetReferenceImage(image1)
    resampler.Update()

    # Threshold the resampled image to create a binary mask
    threshold_filter = itk.BinaryThresholdImageFilter.New(Input=resampler.GetOutput())
    threshold_filter.SetLowerThreshold(lowerThresh)  # Threshold value, adjust if needed
    threshold_filter.SetUpperThreshold(upperThresh)  # Upper threshold value, adjust if needed
    threshold_filter.SetInsideValue(1)
    threshold_filter.SetOutsideValue(0)
    threshold_filter.Update()

    outputFile = "C:/Users/nichmoore/Documents/GitHub/RA4-Project/figures/airtreeResampledSwapped.nii.gz"

    if os.path.exists(outputFile):
        os.remove(outputFile)

    # Save the resampled image as NIfTI
    airtreeResampled = threshold_filter.GetOutput()
    itk.imwrite(airtreeResampled, outputFile)